In [4]:
import sys
!{sys.executable} -m pip install pandas

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [5]:
import sqlite3
import pandas as pd

# create a database connection
conn = sqlite3.connect('healthcare.db')

In [6]:
# create the tables in the database
cursor = conn.cursor()

In [7]:
# read the CSV files into pandas dataframes
benefit_df = pd.read_csv('benefits.csv')
plans_df = pd.read_csv('plans.csv')
rates_df = pd.read_csv('rates.csv')

/var/folders/w4/mky_skrx5fl9dqyyh_0fy5p40000gp/T/ipykernel_57437/3850283623.py:2: DtypeWarning: Columns (9,12,19) have mixed types. Specify dtype option on import or set low_memory=False.
  benefit_df = pd.read_csv('benefits.csv')
/var/folders/w4/mky_skrx5fl9dqyyh_0fy5p40000gp/T/ipykernel_57437/3850283623.py:3: DtypeWarning: Columns (27,64,80,81,82,86,87,88,89,90,91,111,122,123,124,125,147,148) have mixed types. Specify dtype option on import or set low_memory=False.
  plans_df = pd.read_csv('plans.csv')


In [8]:
# Filter By 10 Most Populous States and Drop Unnecessary Columns
top10USStatesByPopulation = ['CA', 'TX', 'FL', 'NY', 'PA', 'IL', 'OH', 'GA', 'NC', 'MI']
# missing states = CA, NY, PA
benefit_df = benefit_df[benefit_df['StateCode'].isin(top10USStatesByPopulation)]
plans_df = plans_df[plans_df['StateCode'].isin(top10USStatesByPopulation)]
rates_df = rates_df[rates_df['StateCode'].isin(top10USStatesByPopulation)]

rates_df_filtered = rates_df.drop(['BusinessYear', 'StateCode', 'IssuerId', 'SourceName', 'ImportDate', 'RateEffectiveDate', 'RateExpirationDate', 'RatingAreaId'], axis=1)
plans_df_filtered = plans_df.drop(['BusinessYear', 'IssuerId', 'SourceName', 'ImportDate', 'PlanBrochure', 'URLForSummaryofBenefitsCoverage', 'HSAOrHRAEmployerContribution', 'HSAOrHRAEmployerContributionAmount', 'InpatientCopaymentMaximumDays', 'NationalNetwork', 'OutOfCountryCoverageDescription', 'OutOfServiceAreaCoverage', 'EHBPediatricDentalApportionmentQuantity', 'DesignType', 'MetalLevel'], axis=1)
benefit_df_filtered = benefit_df.drop(['BusinessYear', 'StateCode', 'IssuerId', 'SourceName', 'ImportDate', 'StandardComponentId', 'CopayInnTier1', 'CopayInnTier2', 'CopayOutofNet', 'CoinsInnTier1', 'CoinsInnTier2', 'CoinsOutofNet', 'Exclusions', 'Explanation', 'EHBVarReason', 'IsExclFromInnMOOP', 'IsExclFromOonMOOP'], axis=1)

In [9]:
# insert the data into the database
rates_df_filtered.to_sql('Rates', conn, if_exists='replace', index=False)
plans_df_filtered.to_sql('Plan', conn, if_exists='replace', index=False)
benefit_df_filtered.to_sql('Benefit', conn, if_exists='replace', index=False)

1113553